In [1]:
import datasets
from datasets import concatenate_datasets
from common.data_utils import get_dataset

In [2]:
train_ds, test_ds = get_dataset(name="yelp_polarity", split_rate=1.0)
train_ds = datasets.Dataset.from_dict(train_ds[:len(train_ds)])
#val_ds = datasets.Dataset.from_dict(val_ds[:len(val_ds)])
#test_ds = datasets.Dataset.from_dict(test_ds[:len(test_ds)])

Dataset yelp_polarity downloaded and prepared to /home/coraline/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/2b33212d89209ed1ea0522001bccc5f5a5c920dd9c326f3c828e67a22c51a98c. Subsequent calls will reuse this data.


In [6]:
train_ds[0]

{'label': 0,
 'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}

In [7]:
import json

In [11]:
output_pth = 'test.json'
test_dict_list = [{1:2}, {3:4}]
json.dump(test_dict_list, open(output_pth, "a"), indent=2)

In [14]:
json.load(open(output_pth, "r"))

[{'1': 2}, {'3': 4}]

In [3]:
from transformers import BertTokenizer
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [4]:
train_encodings = tokenizer(train_ds['text'], truncation=True, padding=True, return_token_type_ids=False, return_tensors='pt')
#test_encodings = tokenizer(test_ds['text'], truncation=True, padding=True, return_token_type_ids=False, return_tensors='pt')

In [5]:
train_ds

Dataset({
    features: ['label', 'text'],
    num_rows: 25000
})

In [6]:
import torch
torch.cuda.empty_cache()

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_ds['label'])
#test_dataset = IMDbDataset(test_encodings, test_ds['label'])

In [7]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
torch.cuda.empty_cache()

t = torch.cuda.get_device_properties(0).total_memory
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
f = c-a  # free inside cache

#GiB, GiB, MiB
print(c*9.31323e-10, a*9.31323e-10, f*9.53674e-7)

0.0 0.0 0.0


In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

t = torch.cuda.get_device_properties(0).total_memory
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
f = c-a  # free inside cache

#GiB, GiB, MiB
print(c*9.31323e-10, a*9.31323e-10, f*9.53674e-7)

0.460937710534656 0.408918090674688 53.268048733184


In [10]:
from transformers import AdamW
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

/home/coraline/anaconda3/envs/bert-attack/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
model.save_pretrained('./data/imdb/saved_model/imdb_bert_base_uncased_finetuned_training')

In [ ]:
from transformers import AdamW
from torch.utils.data import DataLoader

EPOCHS = 10

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

#hyperparameters from BERT authors' recommendations
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * EPOCHS

from transformers import get_linear_schedule_with_warmup
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

#loss_fn = nn.CrossEntropyLoss().to(device)

## Test

In [3]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('../data/imdb/saved_model/imdb_bert_base_uncased_finetuned_normal')
model.eval()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [ ]:
with torch.no_grad():
    

In [5]:
from transformers import BertTokenizer
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [7]:
text_batch = ["this movie is boring."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

outputs = model(input_ids, attention_mask=attention_mask)

In [10]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.5830, -2.3512]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [11]:
outputs[0]

tensor([[ 2.5830, -2.3512]], grad_fn=<AddmmBackward>)

In [30]:
_, preds = torch.max(outputs.logits, dim=1)

In [31]:
preds

tensor([0])